### Moduļi

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from scipy.linalg import expm, logm, polar

### Konstantes

In [ ]:
# Aproksimācijas precizitāte
EPSILON = 1e-4

# Izmantojamo vārtu kopa
GATESET = {}

tempqc = QuantumCircuit(1)
tempqc.t(0)
GATESET[' T'] = tempqc
tempqc = tempqc.copy().inverse()
GATESET['-T'] = tempqc
del tempqc

tempqc = QuantumCircuit(1)
tempqc.h(0)
GATESET[' H'] = tempqc
del tempqc

# Pauli matricas 
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], complex)
Y = np.array([[0, -1j], [1j, 0]], complex)
Z = np.array([[1, 0], [0, -1]], complex)
PAULI_BASE = [X, Y, Z]

In [ ]:
# Pārbauda, vai divas Rz vārti dod pareizo rezultātu
tempqc = QuantumCircuit(1)
tempqc.h(0)
tempqc.p(np.pi / 4, 0)
tempqc.p(np.pi / 4, 0)
print(Operator(tempqc).data)
del tempqc

# Izdrukā vārtu kopu
for gate_name in GATESET:
    print("Gate:", gate_name)
    print(Operator(GATESET[gate_name]).data)

### Saglabātās īsās ķēdes

In [ ]:
# Īsas ķēdes ātrai piekļuvei aproksimācijai
SHORTHAND = []
SHORTHAND_LEN = 16
shorthand_levels = [[] for (_) in range(SHORTHAND_LEN)]

# Sāk ar īsajām ķēdēm ar vienu vārtu
for (gate_name, gate_qc) in GATESET.items():
    gate_U = Operator(gate_qc).data
    shorthand_levels[0].append((gate_name, gate_qc, gate_U))

# Visas iespējamās kombinācijas līdz SHORTHAND_LEN vārtiem
for (i) in range(1, SHORTHAND_LEN):
    for (name, qc, _) in shorthand_levels[i-1].copy():
        for (gate_name, gate_qc) in GATESET.items():
            if (name[-2:] == gate_name == ' H'): # Izvairās no diviem H pēc kārtas (jo H*H=I)
                continue
            if (name[-1] == gate_name[1]) and (name[-2] == '-' or gate_name[0] == '-'): # Izvairās no diviem vienādiem Rz pēc kārtas
                continue
            if (gate_name == name[-2:] == name[-5:-3] == name[-8:-6] == name[-11:-9]): # Izvairās no vairāk kā četrām rotācijām pēc kārtas
                continue
            new_name = name + ' ' + gate_name
            new_qc = qc.copy()
            new_qc.compose(gate_qc, [0], inplace=True)
            new_U = Operator(new_qc).data
            shorthand_levels[i].append((new_name, new_qc, new_U))
    print(f"Pievienoja {len(shorthand_levels[i])} ķēdes garumā {i+1}")

for (i) in range(SHORTHAND_LEN):
    SHORTHAND.extend(shorthand_levels[i])

### Funkcijas

In [ ]:
# Rekursīvais algoritms, kas balstās uz Soloveja-Kitājeva teorēmas
def solovay_kitaev_decomposition(U_target, depth):

    if (depth == 0):
        return shorthand_approximation(U_target)
    
    qc_approx = solovay_kitaev_decomposition(U_target, depth-1)

    if (compare_su2(U_target, Operator(qc_approx).data) < EPSILON):
        return qc_approx

    U_approx = Operator(qc_approx).data
    A, B = gc_decomposition(U_target @ U_approx.conj().T)

    qc_A = solovay_kitaev_decomposition(A, depth-1)
    qc_B = solovay_kitaev_decomposition(B, depth-1)
    qc_A_inv = qc_A.inverse()
    qc_B_inv = qc_B.inverse()

    # Izveido jaunu ķēdi
    qc = QuantumCircuit(1)
    qc.compose(qc_approx, [0], inplace=True)
    qc.compose(qc_B_inv, [0], inplace=True)
    qc.compose(qc_A_inv, [0], inplace=True)
    qc.compose(qc_B, [0], inplace=True)
    qc.compose(qc_A, [0], inplace=True)

    return qc

# Funkcija, kas atrod labāko īso ķēdi dotajam operatoram
# izmanto rekursijas bāzes gadījumā
def shorthand_approximation(U_target):
    print("Ātrais tuvinājums operatoram:")
    print(U_target)

    min_error = float('inf')
    best_entry = SHORTHAND[0]
    for (entry) in SHORTHAND:
        error = compare_su2(U_target, entry[2])
        if (error < min_error):
            min_error = error
            best_entry = entry
    print(f"Tuvākā ķēde {best_entry[0]} ar kļūdu {min_error}")

    return best_entry[1] # atgriež QuantumCircuit

# Grupas komutatora dekompozīcija
def gc_decomposition(U):
    print("\n---GC DEKOMPOZĪCIJA---")
    print("Grupas komutatora dekompozīcija operatoram:")
    print(U)

    print("det U", np.linalg.det(U))

    # Noņem globālo fāzi
    U = remove_global_phase(U)
    print("U bez globālās fāzes:")
    print(U)

    # Izvelk rotācijas asi un leņķi
    U_xs, U_th = extract_axis_angle(U)

    print("angle", U_th)
    print("axis", U_xs)

    # # Versija 1: izmanto phi = sqrt(U_th)
    # phi = np.sqrt(U_th)
    # print("phi", phi)

    # w = [0, 0, 1] # sāk ar Z asi
    # if np.allclose(U_xs, w):
    #     w = [1, 0, 0] # ja U_xs ir Z, izvēlas X asi

    # # Veido ortonormālu bāzi
    # w = np.array(w)
    # u = w - np.dot(w, U_xs) * U_xs
    # u = u / np.linalg.norm(u)
    # v = np.cross(U_xs, u)
    # print("basis u", u)
    # print("basis v", v)

    # # Rotācijas matricas ap u un v asi
    # A = rotation_matrix(u, phi)
    # B = rotation_matrix(v, phi)

    # Versija 2: formula no pētījuma
    phi = 2 * np.arcsin(((1 - np.cos(U_th / 2)) / 2) ** 0.25)
    print("phi", phi)

    X_xs = np.array([1, 0, 0])
    Y_xs = np.array([0, 1, 0])

    # U = S(VWV⁺W⁺)S⁺
    V = rotation_matrix(X_xs, phi)
    print("V:")
    print(V)
    W = rotation_matrix(Y_xs, phi)
    print("W:")
    print(W)
    C = V @ W @ V.conj().T @ W.conj().T
    print("C:")
    print(C)

    # Atrisina vienādojumu S * commutator * S⁺ = U
    K = np.kron(I, C.T) - np.kron(U, I)
    _, _, Vh = np.linalg.svd(K)
    S_vec = Vh.conj().T[:, -1]
    S_raw = S_vec.reshape((2, 2))

    S, _ = polar(S_raw)
    S = remove_global_phase(S)
    print("S:")
    print(S)

    # Pārbauda dekompozīciju
    print("Pārbaude S * commutator * S⁺:")
    print(S @ C @ S.conj().T)

    A = S @ V @ S.conj().T
    B = S @ W @ S.conj().T

    # Tests
    U_reconstructed = A @ B @ A.conj().T @ B.conj().T
    print("Atjaunotais operators no dekompozīcijas:")
    print(U_reconstructed)
    print("Kļūda:")
    print("      ", compare_su2(U, U_reconstructed))
    print("---\n")
    
    return (A, B)

# Funkcija izvelk rotācijas asi un leņķi no unitāra operatora
def extract_axis_angle(R):
    # Konvertē uz SU(2)
    if not np.isclose(np.linalg.det(R), 1, atol=1e-10):
        raise ValueError("Operators nav SU(2) grupa.")

    # Aprēķina leņķi
    trace = np.trace(R)
    theta = np.arccos(np.real(trace) / 2) * 2

    # Ja leņķis ir tuvu nullei, atgriež standarta asi un nulles leņķi
    if np.isclose(theta, 0, atol=1e-12):
        return np.array([1, 0, 0]), 0

    # Aprēķina rotācijas asi
    A = (R - np.cos(theta / 2) * I) / (-1j * np.sin(theta / 2))
    nx = np.real(A[1, 0])
    ny = np.imag(A[1, 0])
    nz = np.real(A[0, 0])
    axis = np.array([nx, ny, nz]) / np.linalg.norm([nx, ny, nz])

    return (axis, theta)

# Funkcija izveido rotācijas matricas no ass un leņķa
def rotation_matrix(axis, theta):
    return expm(-1j * theta / 2 * (axis[0] * X + axis[1] * Y + axis[2] * Z))

# Funkcija noņem globālo fāzi no unitāra operatora
def remove_global_phase(U):
    phase = np.angle(np.linalg.det(U)) / 2
    V = U / np.exp(1j * phase)
    W, _ = polar(V)
    return W

# Funkcija salīdzina divas matricas bez globālās fāzes
def compare_su2(U1, U2):
    U1 = remove_global_phase(U1)
    U2 = remove_global_phase(U2)
    return np.linalg.norm(U1 - U2, 2)

### Ievads

In [ ]:
# TARGET = np.sqrt(0.5) * np.array([
#     [1, 1],
#     [1j, -1j]
# ])
# TARGET = np.array([
#     [0, 1],
#     [1, 0]
# ])
# TARGET = np.array([
#     [np.cos(np.pi / 8), -np.sin(np.pi / 8)],
#     [np.sin(np.pi / 8), np.cos(np.pi / 8)]
# ])
TARGET = np.array([
    [1, 0],
    [0, np.exp(1j * np.pi / 3)]
])
# vai unitāra?
print("Vai unitāra?", np.allclose(TARGET @ TARGET.conj().T, I))
print("Aproksimējamais operators:")
print(TARGET)

### Main bloks

In [ ]:
final_circuit = solovay_kitaev_decomposition(TARGET, 3)

T_su2 = remove_global_phase(TARGET)
print("TARGET bez globālās fāzes:")
print(T_su2)
print()

final_operator = Operator(final_circuit).data
print("Final circuit operator:")
print(final_operator)

print("Final approximation error:", compare_su2(TARGET, final_operator))

final_circuit.draw()